In [ ]:
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform

import tensorflow as tf
import tensorflow_hub as hub

import cv2
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image, ImageOps
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os 

In this Notebook I just wanted to share an attempt to match images with TensorFlow's DELF module.  
So far I mostly saw Kornia attempts, so I hope this might share some new insights.  
This notebook is mostly based on [this tutorial from TensorFlow's page](https://www.tensorflow.org/hub/tutorials/tf_hub_delf_module?hl=en).  
  
To get DELF to work offline, one has to download the compressed file from [HERE](https://tfhub.dev/google/delf/1?tf-hub-format=compressed) , upload it to the input folder and the import using `hub.load(path/to/file)`.

I tried to evaluate the method on the training set but I get poor results. I think there is some problem with how the fundamental matrix is calculated so I think there are still many possibilities for improvement here. I thi

In [ ]:
# setting dry_run = False will cause the notebook not to plot images (e.g. for a submission)
dry_run = False

In [ ]:
def flatten_fundamental_matrix(matrix, precision=8):
    return ' '.join([f'{number:.{precision}f}' for number in matrix.flatten()])

In [ ]:
test = pd.read_csv('../input/image-matching-challenge-2022/test.csv')
batch_ids = list(test['batch_id'])
sample_ids = list(test['sample_id'])

In [ ]:
delf = hub.load('../input/tensorflow-hub-delf').signatures['default']

def run_delf(image):
    np_image = np.array(image)
    float_image = tf.image.convert_image_dtype(np_image, tf.float32)

    return delf(
      image=float_image,
      score_threshold=tf.constant(100.0),
      image_scales=tf.constant([0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0]),
      max_feature_num=tf.constant(1000))

In [ ]:
def match_images(image1, image2, result1, result2, dry_run=False):
    distance_threshold = 0.8

    # Read features.
    num_features_1 = result1['locations'].shape[0]
    print("Loaded image 1's %d features" % num_features_1)

    num_features_2 = result2['locations'].shape[0]
    print("Loaded image 2's %d features" % num_features_2)

    # Find nearest-neighbor matches using a KD tree.
    d1_tree = cKDTree(result1['descriptors'])
    _, indices = d1_tree.query(
      result2['descriptors'],
      distance_upper_bound=distance_threshold)

    # Select feature locations for putative matches.
    locations_2_to_use = np.array([
      result2['locations'][i,]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
      result1['locations'][indices[i],]
      for i in range(num_features_2)
      if indices[i] != num_features_1
    ])
    
    H, inliers = ransac((keypoints_left[matches[:, 0]],
                         keypoints_right[matches[:, 1]]),
                        FundamentalMatrixTransform, min_samples=8,
                        residual_threshold=1, max_trials=5000,
                        random_state=rng)
    
#     H, inliers = cv2.findFundamentalMat(locations_1_to_use, locations_2_to_use, cv2.USAC_MAGSAC, 0.5, 0.999, 100000)
#     fundamental_matrices[sample_id] = flatten_fundamental_matrix(H)

    if dry_run:
        # Visualize correspondences.
        _, ax = plt.subplots()
        inlier_idxs = np.nonzero(inliers)[0]
        plot_matches(
          ax,
          image1,
          image2,
          locations_1_to_use,
          locations_2_to_use,
          np.column_stack((inlier_idxs, inlier_idxs)),
          matches_color='b')
        ax.axis('off')
        ax.set_title('DELF correspondences')

    return H

In [ ]:
fundamental_matrices = {}
for idx, row in test.iterrows():
    
    try:
        sample_id = row['sample_id']
        batch_id = row['batch_id']
        image_1_id = row['image_1_id']
        image_2_id = row['image_2_id']
        img1_path = f'../input/image-matching-challenge-2022/test_images/{batch_id}/{image_1_id}.png'
        img2_path = f'../input/image-matching-challenge-2022/test_images/{batch_id}/{image_2_id}.png'

        img1 = Image.open(img1_path)
        img2 = Image.open(img2_path)

        result1 = run_delf(img1)
        result2 = run_delf(img2)
        
        F = match_images(img1, img2, result1, result2, dry_run = dry_run)
        fundamental_matrices[sample_id] = flatten_fundamental_matrix(F)
        
        if F is not None and F.shape == (3, 3):
            fundamental_matrices[sample_id] = flatten_fundamental_matrix(F)
        else:
            F = np.random.rand(3, 3)
            fundamental_matrices[sample_id] = flatten_fundamental_matrix(F)
    except:
        F = np.random.rand(3, 3)
        fundamental_matrices[sample_id] = flatten_fundamental_matrix(F)

In [ ]:
submission_df = pd.DataFrame.from_dict(fundamental_matrices, orient='index')
submission_df.index = submission_df.index.rename('sample_id')
submission_df = submission_df.rename({0:'fundamental_matrix'},axis=1)
submission_df.to_csv('submission.csv')

In [ ]:
if dry_run:
    !cat submission.csv